In [6]:
import importlib
from pathlib import Path
import json

pth = Path("./")
for typ in ["task", "function"]:
    libs = [
        f"digitalhub_data_dbt.entities.{typ}s.spec",
        f"digitalhub_data_nefertem.entities.{typ}s.spec",
        f"digitalhub_ml_mlrun.entities.{typ}s.spec",
    ]
    (pth / typ).mkdir(parents=True, exist_ok=True)
    
    for lib in libs:
        module = importlib.import_module(lib)
        for i in getattr(module, "spec_registry").values():
            json_schema = i[1].schema_json()
            name = f"schema_{typ}_" + i[1].schema()["title"].lower().replace(f"{typ}params", "") + ".json"
            text = json.dumps(json.loads(json_schema), indent=4)
            (pth / typ /name).write_text(text)